In [14]:
import mmml
import matplotlib.pyplot as plt
import os

import numpy as np
os.environ["XLA_PYTHON_CLIENT_MEM_FRACTION"] = "0.95"
os.environ["CUDA_VISIBLE_DEVICES"] = "1"
import jax
devices = jax.local_devices()
print(devices)
print(jax.default_backend())
print(jax.devices())

[CudaDevice(id=0)]
gpu
[CudaDevice(id=0)]


In [2]:
from pathlib import *

current_path = Path.cwd()
print(current_path)
data_path = current_path / ".." / "data"
data_files = list(data_path.glob("*npz"))
for i in range(len(data_files)):
    print(i, data_files[i])

/home/ericb/mmml
0 /home/ericb/mmml/../data/240124_combined_water.npz
1 /home/ericb/mmml/../data/Data_new.npz
2 /home/ericb/mmml/../data/esp13415.npz
3 /home/ericb/mmml/../data/esp2000.npz
4 /home/ericb/mmml/../data/md17_ethanol.npz
5 /home/ericb/mmml/../data/mulitpoles1000.npz
6 /home/ericb/mmml/../data/mulitpoles10000.npz
7 /home/ericb/mmml/../data/mulitpoles766000.npz
8 /home/ericb/mmml/../data/qm9-combined.npz
9 /home/ericb/mmml/../data/qm9-esp-dip-1000-0.npz
10 /home/ericb/mmml/../data/qm9-esp-dip-1000-1.npz
11 /home/ericb/mmml/../data/qm9-esp-dip-1000-2.npz
12 /home/ericb/mmml/../data/qm9-esp-dip-1000-3.npz
13 /home/ericb/mmml/../data/qm9-esp-dip-40000-0.npz
14 /home/ericb/mmml/../data/qm9-esp-dip-40000-1.npz
15 /home/ericb/mmml/../data/qm9-esp-dip-40000-2.npz
16 /home/ericb/mmml/../data/qm9-esp-dip-6907-3.npz
17 /home/ericb/mmml/../data/qm9-esp10000.npz


In [3]:
index = 3
data_path_resolved = data_files[index].resolve()
data_loaded = np.load(data_path_resolved, 
allow_pickle=True)

In [4]:
# for k in data_loaded.keys():
#     print(k)
#     shape = data_loaded[k].shape
#     print(shape
#     )
#     if len(shape) < 3:
#         plt.hist(data_loaded[k])
#         plt.show()

In [5]:
from mmml import dcmnet

In [6]:
import jax

from mmml.dcmnet.dcmnet.data import prepare_datasets
from mmml.dcmnet.dcmnet.modules import MessagePassingModel
from mmml.dcmnet.dcmnet.training import train_model, train_model_dipo

key = jax.random.PRNGKey(0)


In [7]:
model = MessagePassingModel(
    features=16, max_degree=2, num_iterations=2,
    num_basis_functions=16, cutoff=4.0, n_dcm=1,
    include_pseudotensors=False,
)

In [8]:
prepare_datasets?

Signature:
prepare_datasets(
    key,
    num_train,
    num_valid,
    filename,
    clean=False,
    esp_mask=False,
    clip_esp=False,
    natoms=60,
)
Docstring:
Prepare datasets for training and validation.

Wrapper function that calls prepare_multiple_datasets and then
creates train/validation splits and dictionaries.

Parameters
----------
key : jax.random.PRNGKey
    Random key for dataset shuffling
num_train : int
    Number of training samples
num_valid : int
    Number of validation samples
filename : str or list
    Filename(s) to load datasets from
clean : bool, optional
    Whether to filter failed calculations, by default False
esp_mask : bool, optional
    Whether to create ESP masks, by default False
clip_esp : bool, optional
    Whether to clip ESP to first 1000 points, by default False
natoms : int, optional
    Maximum number of atoms per system, by default 60

Returns
-------
tuple
    A tuple containing train_data and valid_data dictionaries
File:      ~/mmml/mmm

In [9]:
train_data, valid_data = prepare_datasets(
    key, num_train=1200, num_valid=100,
    filename=[data_path_resolved],
    clean= False, esp_mask=True,
    # clip_esp=True,
)




shape (2000, 60, 3)
R (2000, 60, 3)
(2000, 60, 3)
['R', 'Z', 'N', 'mono', 'esp', 'vdw_surface', 'n_grid', 'espMask']
2000
0 R 2000 (2000, 60, 3)
1 Z 2000 (2000, 60)
2 N 2000 (2000, 1)
3 mono 2000 (2000, 60)
4 esp 2000 (2000, 4953)
5 vdw_surface 2000 (2000, 4953, 3)
6 n_grid 2000 (2000,)
7 espMask 2000 (2000, 4953)


In [10]:
esp_data = train_data["esp"]
esp_data.shape

(1200, 4953)

In [11]:
train_model?

Signature:     
train_model(
    key,
    model,
    train_data,
    valid_data,
    num_epochs,
    learning_rate,
    batch_size,
    writer,
    ndcm,
    esp_w=1.0,
    restart_params=None,
    *,
    loss_step_fn: Callable = <PjitFunction of <function train_step at 0x7df3d25efe20>>,
    eval_step_fn: Callable = <PjitFunction of <function eval_step at 0x7df3d220e8c0>>,
    optimizer_fn: Callable = <function <lambda> at 0x7df3d220fd00>,
    use_ema: bool = True,
    ema_decay: float = 0.999,
    use_grad_clip: bool = False,
    grad_clip_norm: float = 2.0,
    log_extra_metrics: Optional[Callable] = <function _log_extra_metrics_none at 0x7df3d220fbe0>,
    save_best_params_with_ema: bool = True,
    extra_valid_args: dict = None,
    extra_train_args: dict = None,
)
Call signature: train_model(*args, **kwargs)
Type:           partial
String form:    functools.partial(<function train_model_general at 0x7df3d220fb50>, loss_step_fn=<PjitFunction of <...> tra_metrics=<function _log_extr

In [12]:
params, valid_loss = train_model(
    key=key, model=model,
    writer=None,
    train_data=train_data, valid_data=valid_data,
    num_epochs=10, learning_rate=1e-3, batch_size=1,
    ndcm=model.n_dcm, esp_w=10000.0,
)

Preparing batches
..................
sparse_pairwise_indices
sparse_pairwise_indices


AttributeError: 'NoneType' object has no attribute 'add_scalar'